In [25]:
from astropy.table import Table
import os

import matplotlib.pyplot as plt

In [67]:
filepath_catalog = 'A3D_ETGsLTGs.votable'
catalog          = Table.read(filepath_catalog,format='votable')
catalog

Galaxy,RA,DEC,SBF,NED-D,Virgo,VHel,D,M_K,A_B,T-type,log_Re_
str10,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64
IC0065,15.230966,47.681984,0,0,0,2614,39.7,-23.54,0.64,4.0,1.38
IC0163,27.312431,20.711317,0,0,0,2749,39.7,-22.38,0.36,8.0,1.35
IC0239,39.11625,38.97,0,0,0,903,15.7,-22.23,0.31,6.0,1.9
IC0540,142.542755,7.902529,0,0,0,2035,30.0,-21.89,0.26,3.5,1.08
IC0591,151.865479,12.27452,0,0,0,2839,41.2,-21.82,0.15,6.0,1.19
IC0610,156.618179,20.228252,0,0,0,1170,19.6,-21.53,0.09,3.9,1.02
IC0750,179.717606,42.722404,0,1,0,701,36.8,-24.71,0.09,2.1,1.24
IC0777,184.849228,28.309881,0,0,0,2541,39.0,-21.92,0.1,2.6,0.95
IC0800,188.486313,15.354542,0,0,0,2326,35.8,-22.2,0.16,5.2,1.56


In [83]:
def sh_download_cutout(RA,DEC,name,survey,height,width,FOV,projection,coordsys,rotation_angle,format,filename_sh):
    """
    Writes a .sh file calling Aladin to retrieve cutouts of a target galaxy.
    Input: catalog (astropy table): table containing the galaxies of interest (name, RA, DEC)
        RA (float): RA coordinates of the target galaxy (in degrees)
        DEC (float): DEC coordinates of the target galaxy (in degrees)
        name (str): name of the target galaxy
        survey (str): name of the image survey
        height (int): height in pixels of the cutout image
        DEC (float): height in pixels of the cutout image
        width (int): width in pixels of the cutout image
        FOV (float): Field of View of the cutout in degrees
        projection (str): projection to use for the cutout
        coordsys (str): coordinate system to use for the cutout
        rotation_angle (float): rotation of the cutout 
        format (str): format of the cutout
        filename_sh (str): name of the output .sh file that will be created
    Output: nothing is returned, only the file is written 
    """
    URL_BASE = "https://alasky.cds.unistra.fr/hips-image-services/hips2fits?hips="
    AladinURL = '"' +URL_BASE+survey+'&width='+width+'&height='+height+'&fov='+FOV+'&projection='+projection+'&coordsys='+coordsys+'&rotation_angle='+rotation_angle+'&object='+RA+'+'+DEC+'&format='+format+'"'+'\n'
    with open(filename_sh,'a') as f:
        f.write(
            '#!/bin/bash \n'+ 'AladinURL='+ AladinURL +\
            'wget -O '+name+'.'+format +' "$AladinURL" \n'      
        )
    return 0

def get_several_sh_download_cutouts(catalog,survey,height=512,width=512,FOV=0.5,projection='SIN',coordsys='icrs',rotation_angle=0.0,format='png',filename_sh='download_skyview.sh'):
    """
    Write the .sh that will be used to call Aladin to retrieve curtouts of several galaxies from an input catalog containg the names of the objects and the RA,DEC coordinates.
    Input: catalog (astropy table): table containing the galaxies of interest (name, RA, DEC)
           survey (str): name of the image survey
           height (int): height in pixels of the cutout image
           width (int): width in pixels of the cutout image
           FOV (float): Field of View of the cutout in degrees
           projection (str): projection to use for the cutout
           coordsys (str): coordinate system to use for the cutout
           rotation_angle (float): rotation of the cutout 
           format (str): format of the cutout
           filename_sh (str): name of the output .sh file that will be created
    Output: nothing is returned, only the file is written  
    """
    # Delete .sh file if already exists
    if os.path.exists(filename_sh):
        os.remove(filename_sh)
    for i in range(len(catalog)):
        ra   = catalog[i]['RA']
        dec  = catalog[i]['DEC']
        name = catalog[i]['Galaxy']
        adaptive_FOV  = (10**(catalog[i]['log_Re_'])/3600)*10
        sh_download_cutout(str(ra),str(dec),name,survey,str(height),str(width),str(adaptive_FOV),projection,coordsys,str(rotation_angle),format,filename_sh)
    return 0

In [79]:
## For a single cutout
#sh_download_cutout(RA="148.8882193985",DEC="+69.0652951404",name='M81',survey="CDS%2FP%2FDSS2%2Fcolor",height='512',width='512',FOV='0.8',projection='SIN',coordsys='icrs',rotation_angle='0.0',format='jpg')

In [84]:
## For an entire catalog
get_several_sh_download_cutouts(catalog,survey='CDS%2FP%2FDSS2%2Fr',height=512,width=512,FOV=0,projection='SIN',coordsys='icrs',rotation_angle=0.0,format='fits',filename_sh='download_A3D_skyview.sh')

0